# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Data Preprocessing

In [2]:
data = pd.read_csv('titanic.csv')

In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
#checking for null values
nullValues = data.isnull().sum()
nullValues

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
#deleting all the columns whose missing values are over 20%
delCols = ['PassengerId', 'Name', 'Ticket', 'Fare']
for i, v in nullValues.items():
    if (v/data.shape[0]) > 0.2:
        delCols.append(i)
data.drop(delCols, axis = 1, inplace = True)

In [6]:
#replacing male and female to 0 and 1
data['Sex'].replace({'male' : 0, 'female' : 1}, inplace = True)

In [7]:
#one hot encoding passenger class
data = pd.get_dummies(data, columns = ['Pclass'])

In [8]:
#seperating labels and attributes
X = data.drop('Survived', axis = 1)
y = data['Survived']

In [9]:
#seperating the data into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [10]:
#creating a function to fill in the missing values with the most common values
def fillMissing(df, ref): 
    #df is the dataframe in which we'll fill in the values
    #ref is the reference dataframe
    missingItems = df.isnull().sum()
    missingCols = []
    r = ref.mode()
    for i, v in missingItems.items():
        if v != 0:
            missingCols.append(i)
    for col in missingCols:
        df[col] = df[col].fillna(r[col][0])

In [11]:
#filling in the missing values in X_train and X_val using X_train properties
fillMissing(X_train, X_train)
fillMissing(X_val, X_train)

In [12]:
#one hot encoding
X_train = pd.get_dummies(X_train)
X_val = pd.get_dummies(X_val)

In [13]:
#function to convert numerical columns to one hot encoded columns
def convertToCat(X, name, space):
    minimum = min(X[name])
    maximum = max(X[name])
    i = 0
    while i < maximum:
        v = i + space
        colName = name + " " + str(i) + "-" + str(v)
        array = np.where(((X[name] >= i) & (X[name] < v)), 1, 0)
        X[colName] = array
        i += space
    X.drop(name, axis = 1, inplace = True)

In [14]:
#one hot encoding numerical columns for both training and validation set
convertToCat(X_train, 'Age', 10)
convertToCat(X_train, 'Parch', 1)
convertToCat(X_train, 'SibSp', 1)
convertToCat(X_val, 'Age', 10)
convertToCat(X_val, 'Parch', 1)
convertToCat(X_val, 'SibSp', 1)

In [15]:
#balancing columns for validation set
for i in X_train.columns:
    if i not in X_val.columns:
        X_val[i] = 0

#balancing columns for training set
for i in X_val.columns:
    if i not in X_train.columns:
        X_train[i] = 0

In [16]:
X_train.head()

,Sex,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Age 0-10,Age 10-20,Age 20-30,...,Parch 4-5,Parch 5-6,SibSp 0-1,SibSp 1-2,SibSp 2-3,SibSp 3-4,SibSp 4-5,SibSp 5-6,SibSp 6-7,SibSp 7-8
331,0,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
733,0,0,1,0,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0
382,0,0,0,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
704,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,0
813,1,0,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,0


In [17]:
X_val.head()

,Sex,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Age 0-10,Age 10-20,Age 20-30,...,Parch 4-5,SibSp 0-1,SibSp 1-2,SibSp 2-3,SibSp 3-4,Parch 5-6,SibSp 4-5,SibSp 5-6,SibSp 6-7,SibSp 7-8
709,0,0,0,1,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
439,0,0,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
840,0,0,0,1,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,0,0
720,1,0,1,0,0,0,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
39,1,0,0,1,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0


# Modelling

In [18]:
#Node class for implementing the tree
class Node():
    def __init__(self, left = None, right = None, param = None, val = None):
        self.left = left #points to the left node
        self.right = right #points to the right node
        self.param = param #stores the current node's param that needs to be checked
        self.val = val #stores either 0 or 1 only when this node is a leaf

In [19]:
#Implementing DecisionTree using OOPS
class DecisionTree():
    def __init__(self):
        self.root = None #root node
        self.pred = None #predictions
    
    def pure(self, y):
        return len(np.unique(y)) == 1 #check if y is pure
    
    def gini(self, y):
        #helper function to calculate gini index of a given node
        if self.pure(y):
            return 0 #return 0 if the leaf is pure
        yValues = y.value_counts()
        prob1 = yValues[1] / (yValues[1] + yValues[0])
        prob0 = yValues[0] / (yValues[1] + yValues[0])
        return (1 - (prob1 ** 2) - (prob0 ** 2)) #return gini index using this formula
    
    def getGini(self, X, y):
        #returns the weighted average of the given gini index
        g1, g0 = 0, 0
        yX1 = y[X == 1]
        yX0 = y[X == 0]
        #get gini impurity for yX1
        if len(yX1) != 0:
            g1 = self.gini(yX1)
        #get gini imputiy for yX2
        if len(yX0) != 0:
            g0 = self.gini(yX0)
        firstTerm = (len(yX1) / len(y)) * g1 #weighted average of 1 class
        secondTerm = (len(yX0) / len(y)) * g0 #weighted average of 0 class
        return firstTerm + secondTerm #return the overall weighted average
    
    def getInfoGain(self, X, y):
        #calculate H(Y)
        firstTerm, secondTerm = 0, 0
        p1 = y.sum() / len(y)
        p0 = 1 - p1
        if p1 != 0:
            firstTerm = p1 * np.log(p1)
        if p0 != 0:
            secondTerm = p0 * np.log(p0)
        Hy = -(firstTerm + secondTerm)
        
        #calculate P(X1) and P(X0)
        pX1 = X.sum() / len(X)
        pX0 = 1 - pX1
        
        #calculate H(Y|X1)
        #step 1: for X == 1
        innerFirstTerm, innerSecondTerm = 0, 0
        y1 = y[X == 1]
        if len(y1) != 0:
            y1True = y1.sum() / len(y1)
            y1False = 1 - y1True
            if y1True != 0:
                innerFirstTerm = y1True * np.log(y1True)
            if y1False != 0:
                innerSecondTerm = y1False * np.log(y1False)
        firstTerm = -pX1 * (innerFirstTerm + innerSecondTerm)
        
        #step 2: for X == 0
        innerFirstTerm, innerSecondTerm = 0, 0
        y0 = y[X == 0]
        if len(y0) != 0:
            y0True = y0.sum() / len(y0)
            y0False = 1 - y0True
            if y0True != 0:
                innerFirstTerm = y0True * np.log(y0True)
            if y0False != 0:
                innerSecondTerm = y0False * np.log(y0False)
        secondTerm = -pX0 * (innerFirstTerm + innerSecondTerm)
        
        Hyx = firstTerm + secondTerm
        
        return Hy - Hyx #return the information gain
    
    def fit(self, X, y, t = 'gini'):
        #fit function calls the helper __fit__ function
        self.root = self.__fit__(X, y, t = t)
    
    def __fit__(self, X, y, root = None, t = 'gini'):
        #helper function for training
        if self.pure(y):
            #if the node is pure, make it a leaf and assign it with the corresponding value
            return Node(param = 'leaf', val = y.iloc[0])
        if len(X.columns) == 0:
            #if no more splits are possible, classify the node as a leaf and assign the majority value of y as it's value
            val = y.value_counts().index[0]
            return Node(param = 'leaf', val = val)
        colName = None
        
        #calculate gini impurity for all the features
        if t == 'gini':
            gi = 10
            for i in X.columns:
                g = self.getGini(X[i], y) #get gini index of the current feature
                #update parameters when the new gini index is smaller
                if g < gi:
                    gi = g
                    colName = i
            if gi == 0:
                #if gini index is 0, then the leaf is pure
                return Node(param = 'leaf', val = y.iloc[0])
            
        #calculate information gain for all the features
        elif t == 'ig':
            infoGain = -1
            for i in X.columns:
                ig = self.getInfoGain(X[i], y) #get information gain of the current feature
                #update parameters when the new information gain is more
                if ig > infoGain:
                    infoGain = ig
                    colName = i
        
        node = Node(param = colName) #create a new node with given params

        #drop the current column
        XNew = X.drop(colName, axis = 1)

        #to the left node we pass X[colName == 1]
        left = X[colName] == 1
        XLeft = XNew[left]
        yLeft = y[left]
        if len(yLeft) == 0:
            #if we can't split the tree any further, assign a leaf node with most probable value of y
            val = y.value_counts().index[0]
            return Node(param = 'leaf', val = val)
        node.left = self.__fit__(XLeft, yLeft, node, t) #recurse to the left
        
        #to the right node we pass X[colName == 0]
        right = X[colName] == 0
        XRight = XNew[right]
        yRight = y[right]
        if len(yRight) == 0:
            #if we can't split the tree any further, assign a leaf node with most probable value of y
            val = y.value_counts().index[0]
            return Node(param = 'leaf', val = val)
        node.right = self.__fit__(XRight, yRight, node, t) #recurse to the right
        
        return node
    
    def predict(self, X):
        #prediction method
        self.pred = []
        for i in range(len(X)):
            #traverse through each example
            a = self.root
            while a.param != 'leaf':
                #traverse through the tree till we find a leaf node and get the prediction
                if X.iloc[i][a.param] == 1:
                    a = a.left
                else:
                    a = a.right
            self.pred.append(a.val)
        return self.pred #return the predictions

# Training using Gini Index

In [20]:
gini = DecisionTree()
gini.fit(X_train, y_train, t = 'gini')
print("Performance on training set : ")
print()
giniPred = gini.predict(X_train)
print(classification_report(y_train, giniPred))

Performance on training set : 

              precision    recall  f1-score   support

           0       0.82      0.97      0.88       444
           1       0.92      0.64      0.75       268

    accuracy                           0.84       712
   macro avg       0.87      0.80      0.82       712
weighted avg       0.85      0.84      0.84       712



# Training using Information Gain

In [21]:
infoGain = DecisionTree()
infoGain.fit(X_train, y_train, t = 'ig')
print("Performance on training set : ")
print()
igPred = infoGain.predict(X_train)
print(classification_report(y_train, igPred))

Performance on training set : 

              precision    recall  f1-score   support

           0       0.85      0.97      0.91       444
           1       0.94      0.72      0.82       268

    accuracy                           0.88       712
   macro avg       0.90      0.85      0.86       712
weighted avg       0.89      0.88      0.87       712



# Evaluating performace of Gini Decision tree

In [22]:
predGini = gini.predict(X_val)
print(classification_report(y_val, predGini))

              precision    recall  f1-score   support

           0       0.79      0.90      0.84       105
           1       0.83      0.65      0.73        74

    accuracy                           0.80       179
   macro avg       0.81      0.78      0.78       179
weighted avg       0.80      0.80      0.79       179



# Evaluating performace of Information Gain Decision tree

In [23]:
predIG = infoGain.predict(X_val)
print(classification_report(y_val, predIG))

              precision    recall  f1-score   support

           0       0.81      0.90      0.85       105
           1       0.83      0.70      0.76        74

    accuracy                           0.82       179
   macro avg       0.82      0.80      0.80       179
weighted avg       0.82      0.82      0.81       179



# Best accuracy on the test data was obtained using Information Gain

In [24]:
print(classification_report(y_val, predIG))

              precision    recall  f1-score   support

           0       0.81      0.90      0.85       105
           1       0.83      0.70      0.76        74

    accuracy                           0.82       179
   macro avg       0.82      0.80      0.80       179
weighted avg       0.82      0.82      0.81       179

